## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2023, 11:59pm

#### Firstname Lastname: Giulio Duregon

#### E-mail: gjd9961@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [49]:
%%writefile sol08pr01.py
#Imports
import numpy as np
from mpi4py import MPI

# Initialize comm network, get rank so processes can identify themselves / execute as appropriate
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
info = MPI.Status()

# Get total number of processes
size = comm.Get_size() 
last_rank = size-1

# Initialize buffer
buffer = np.zeros(1)

# Start of with the first process rank == 0
if rank == 0:
    print(f"Process {rank}, sending number: {buffer[0]}, to Process {rank+1}")
    comm.Send(buffer, dest=(rank+1))
    
    # Wait for message from last process
    print(f"Waiting for value from Process {size-1}")
    comm.Recv(buffer, source = (last_rank))
    
    # Print final output
    print(f"Received Final Number From Process {last_rank}: {buffer[0]}")
    
# Do all for N-2
if rank != last_rank and rank != 0:
    # Receive value into buffer
    comm.Recv(buffer, source=(rank-1), status=info)
    print(f"Process {rank}, received the number {buffer[0]}, from Process {info.Get_source()}")
    # Square rank and add to buffer
    buffer += rank**2
    
    # Send value
    comm.Send(buffer, dest=(rank+1))
    
# Check for the final process (N-1)
if rank == last_rank:
    comm.Recv(buffer, source=(last_rank-1), status=info)
    print(f"Process {rank}, received the number {buffer[0]}, from Process {info.Get_source()}")
    # Square rank and add to buffer
    buffer += rank**2
    
    # Send value, but this time to process 0
    print("Sending result back to Process 0")
    comm.Send(buffer, dest=0)
    

Overwriting sol08pr01.py


In [50]:
!mpiexec --oversubscribe -n 10 python sol08pr01.py

Process 0, sending number: 0.0, to Process 1
Waiting for value from Process 9
Process 1, received the number 0.0, from Process 0
Process 2, received the number 1.0, from Process 1
Process 3, received the number 5.0, from Process 2
Process 4, received the number 14.0, from Process 3
Process 5, received the number 30.0, from Process 4
Process 6, received the number 55.0, from Process 5
Process 7, received the number 91.0, from Process 6
Process 8, received the number 140.0, from Process 7
Process 9, received the number 204.0, from Process 8
Sending result back to Process 0
Received Final Number From Process 9: 285.0
--------------------------------------------------------------------------
A system call failed during shared memory initialization that should
not have.  It is likely that your MPI job will now either abort or
experience performance degradation.

  Local host:  Giulios-MBP.lan
  System call: unlink(2) /var/folders/rk/rwsr6gss0vz3g4fz3_kt0x0m0000gn/T//ompi.Giulios-MBP.501/pid

---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [91]:
%%writefile sol08pr02.py
#Imports
import numpy as np
from mpi4py import MPI
import argparse

# Initialize comm network, get rank so processes can identify themselves / execute as appropriate
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
info = MPI.Status()

# Get total number of processes
size = comm.Get_size() 
last_rank = size-1

# Initialize buffer for each process
p = np.ones(1)

# Process 0 Logic
if rank == 0:   
    
    # Using arg parse rather than sys.stdin 
    # Easier to prove code works in Jupyter Notebook / Run from CLI
    parser = argparse.ArgumentParser(
                    prog='ProgramName',
                    description='What the program does',
                    epilog='Text at the bottom of help')

    parser.add_argument('nums', type=float, nargs=2,
                        help='stores 2 numbers')
    args = parser.parse_args()
    
    # Retrieve x,y from CLI
    x,y = args.nums
    
    # Check for x,y != 0
    if (x == 0.0) or (y == 0.0):
        raise AssertionError(f"Inputs must be non zero! x={x}, y={y}")
    
    # Make values into arrays to send
    x = np.array(x)
    y = np.array(y)
    
    # Send y to Process 1
    comm.Send(y, dest=1)
        
    # Mulitply p * x, send to Process 1
    for i in range(5):
        p *= x
        print(f"Sending message {i}, p={p}")
        comm.Send(p, dest=1)
        comm.Recv(p, source=1)
    print(f"Final Value of p={p[0]}")
    
        
# Process 1 Logic
if rank == 1:
    
    # Receive y from Process 0
    y = np.ones(1)
    comm.Recv(y, source=0)
    
    # Receive p from Process 0
    for i in range(1,10,2):
        comm.Recv(p, source=0)
        
        # Divide p by y
        p /= y
        # Send back to Process 0
        print(f"Sending message {i}, p={p}")
        comm.Send(p,dest=0)

Overwriting sol08pr02.py


In [85]:
!mpiexec -n 2 python sol08pr02.py 2 4

Sending message 0, p=[2.]
Sending message 1, p=[0.5]
Sending message 1, p=[1.]
Sending message 3, p=[0.25]
Sending message 2, p=[0.5]
Sending message 5, p=[0.125]
Sending message 3, p=[0.25]
Sending message 7, p=[0.0625]
Sending message 4, p=[0.125]
Sending message 9, p=[0.03125]
Final Value of p=0.03125


In [86]:
!mpiexec -n 2 python sol08pr02.py 1 3 

Sending message 0, p=[1.]
Sending message 1, p=[0.33333333]
Sending message 1, p=[0.33333333]
Sending message 3, p=[0.11111111]
Sending message 2, p=[0.11111111]
Sending message 5, p=[0.03703704]
Sending message 3, p=[0.03703704]
Sending message 7, p=[0.01234568]
Sending message 4, p=[0.01234568]
Sending message 9, p=[0.00411523]
Final Value of p=0.004115226337448559


In [87]:
!mpiexec -n 2 python sol08pr02.py 5 7 

Sending message 0, p=[5.]
Sending message 1, p=[0.71428571]
Sending message 1, p=[3.57142857]
Sending message 3, p=[0.51020408]
Sending message 2, p=[2.55102041]
Sending message 5, p=[0.36443149]
Sending message 3, p=[1.82215743]
Sending message 7, p=[0.2603082]
Sending message 4, p=[1.30154102]
Sending message 9, p=[0.18593443]
Final Value of p=0.1859344320818706


In [88]:
!mpiexec -n 2 python sol08pr02.py 5 10

Sending message 0, p=[5.]
Sending message 1, p=[0.5]
Sending message 1, p=[2.5]
Sending message 3, p=[0.25]
Sending message 2, p=[1.25]
Sending message 5, p=[0.125]
Sending message 3, p=[0.625]
Sending message 7, p=[0.0625]
Sending message 4, p=[0.3125]
Sending message 9, p=[0.03125]
Final Value of p=0.03125
